<a href="https://colab.research.google.com/github/Meyyaou/pythondaily/blob/main/STEP4-Webscrape/webscrape_project_(minione).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#import all tools
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml
import re

In [55]:
#urls=[]
#urls.append("https://www.amazon.fr/Dragon-Ball-Super-Akira-Toriyama/dp/234406446X/ref=sr_1_3?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-3")
#urls.append("https://www.amazon.fr/Carnets-lapothicaire-T13-Natsu-Hyuuga/dp/B0CW5W4CMQ/ref=sr_1_4?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-4")
#urls.append("https://www.amazon.fr/Solo-Leveling-DUBU-REDICE-STUDIO/dp/2382882557/ref=sr_1_5?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-5")
#urls.append("https://www.amazon.fr/Dragon-Ball-Full-Color-Lenfance/dp/2344061568/ref=sr_1_6?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-6")
#urls.append("https://www.amazon.fr/Dragon-Ball-Full-Color-Lenfance/dp/2344061576/ref=sr_1_8?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-8")
#urls.append("https://www.amazon.fr/One-Punch-Man-29-One/dp/B0CW5RB58G/ref=sr_1_9?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-9")
#urls.append("https://www.amazon.fr/Berserk-42-Kentaro-Miura/dp/2344063641/ref=sr_1_10?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-10")
#urls.append("https://www.amazon.fr/One-Piece-Coffret-Marine-Tomes/dp/2344062688/ref=sr_1_11?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-11")
#urls.append("https://www.amazon.fr/Tokyo-Revengers-31-Coffret-collector/dp/2344061800/ref=sr_1_13?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-13")
#urls.append("https://www.amazon.fr/Jujutsu-Kaisen-T24-Gege-Akutami/dp/B0CW5Q9PH6/ref=sr_1_14?content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&dib=eyJ2IjoiMSJ9.kIDCwAetKapTRz2B-l_-WkqvUMs8DhBa5-HbF_QML4fILA3GQGnFIK41u_VDvjCs973vxX3jk_zma7gpNzMW3ixQzEYiDFC4LhRoVmHTiUz-8sgEDXDom4BRfMZNJHO_uLXqk1v-nh2zQ3CMIH8IRLtBy99zPX8PRumLaILRC0mVWRJ7s9PZ0gyXhncJrk5cvLWJpIxZ8IMrktEcurwrNvroypFv0fb0qHmxhWyFOUA.vP3gdHKlVcCeiQihNSQs4oJhZhzpJRxMpY1ErtVS3K8&dib_tag=se&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&qid=1720620825&refinements=p_72%3A184904031&s=books&sr=1-14")
url="https://www.amazon.fr/s?rh=n%3A302004%2Cp_72%3A4-&content-id=amzn1.sym.6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pd_rd_r=adfe474f-e4a1-4cdd-b4f5-8f9b5bf26580&pd_rd_w=QVxJ8&pd_rd_wg=TnXxt&pf_rd_p=6dd8bd97-fb7f-4645-b144-27f8255dbcd8&pf_rd_r=F16SHWEG7V4Z3YQ0W5B6&ref=Oct_d_otopr_S"

In [54]:
page=requests.get(url)
#page

In [56]:
content=page.text
#content

In [22]:
soup=BeautifulSoup(content, 'lxml')

In [58]:
#to get readable html post parsing
#soup.prettify()

In [59]:
for header in soup.find_all('header'):
  header.decompose()
soup.find_all('header')

[]

In [60]:
#get the title
titles=[]
pattern =re.compile(r'\d+')

for title in soup.find_all('span', class_='a-text-normal'):
  if not pattern.match(title.string):
    titles.append(title.string)
#titles

In [63]:
#get auteur
auteurs=[]

for auteur in soup.find_all('span', class_='a-size-base'):
  auteurs.append(auteur.string)
#auteurs

In [ ]:
#get date de sortie

In [ ]:
#get price

In [ ]:
#get price livraison

In [ ]:
#get price kindle

In [ ]:
#get ordre dans le top!